In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pancreatic_Cancer"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Pancreatic_Cancer/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Pancreatic_Cancer/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Pancreatic_Cancer/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Pancreatic_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Find the directory corresponding to Pancreatic Cancer
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)

# Find the directory that matches our trait: Pancreatic_Cancer
pancreatic_dirs = [dir_name for dir_name in tcga_dirs if "pancreatic" in dir_name.lower()]

if not pancreatic_dirs:
    print(f"No matching directory found for trait: {trait}")
    # Record that this trait is not available and exit
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
else:
    # Select the most relevant directory
    selected_dir = pancreatic_dirs[0]  # Should be 'TCGA_Pancreatic_Cancer_(PAAD)'
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Get file paths for clinical and genetic data
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    # Step 3: Load the files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # Step 4: Print column names of clinical data
    print("Clinical data columns:")
    print(clinical_df.columns.tolist())


Clinical data columns:
['CDE_ID_3226963', '_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'adenocarcinoma_invasion', 'age_at_initial_pathologic_diagnosis', 'alcohol_history_documented', 'alcoholic_exposure_category', 'amount_of_alcohol_consumption_per_day', 'anatomic_neoplasm_subdivision', 'anatomic_neoplasm_subdivision_other', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_diabetes_onset', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'days_to_pancreatitis_onset', 'family_history_of_cancer', 'followup_case_report_form_submission_reason', 'followup_treatment_success', 'form_completion_date', 'frequency_of_alcohol_consumption', 'gender', 'histologic_grading_tier_category', 'histolog

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate age and gender columns
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# First check if the directory exists and what subdirectories are available
import os
print(f"TCGA root directory exists: {os.path.exists(tcga_root_dir)}")
if os.path.exists(tcga_root_dir):
    print("Available directories:", os.listdir(tcga_root_dir))

# Using the full directory name for pancreatic cancer
tcga_dir = os.path.join(tcga_root_dir, "TCGA_Pancreatic_Cancer_(PAAD)")

# Check if the directory exists
if os.path.exists(tcga_dir):
    clinical_file_path, _ = tcga_get_relevant_filepaths(tcga_dir)
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')

    # Extract and preview age columns
    if candidate_age_cols:
        age_preview = {}
        for col in candidate_age_cols:
            if col in clinical_df.columns:
                age_preview[col] = clinical_df[col].head(5).tolist()
        print("Age columns preview:")
        print(age_preview)

    # Extract and preview gender columns
    if candidate_gender_cols:
        gender_preview = {}
        for col in candidate_gender_cols:
            if col in clinical_df.columns:
                gender_preview[col] = clinical_df[col].head(5).tolist()
        print("Gender columns preview:")
        print(gender_preview)
else:
    print(f"Directory not found: {tcga_dir}")
    print("Please ensure the correct directory structure and naming convention.")


TCGA root directory exists: True
Available directories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)'

### Step 3: Select Demographic Features

In [4]:
# Examining the age columns
age_col = None
if 'age_at_initial_pathologic_diagnosis' in {'age_at_initial_pathologic_diagnosis': [65, 48, 75, 71, 70], 'days_to_birth': [-23962, -17794, -27600, -26028, -25920]}:
    # This column has appropriate values directly representing age in years
    age_col = 'age_at_initial_pathologic_diagnosis'
elif 'days_to_birth' in {'age_at_initial_pathologic_diagnosis': [65, 48, 75, 71, 70], 'days_to_birth': [-23962, -17794, -27600, -26028, -25920]}:
    # This column has negative values representing days since birth (can be converted to years)
    age_col = 'days_to_birth'

# Examining the gender columns
gender_col = None
if 'gender' in {'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'FEMALE']} and len({'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}) > 0:
    # This column has appropriate gender values
    gender_col = 'gender'

# Print chosen columns
print(f"Chosen age column: {age_col}")
print(f"Chosen gender column: {gender_col}")


Chosen age column: age_at_initial_pathologic_diagnosis
Chosen gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Define the cohort directory
selected_dir = 'TCGA_Pancreatic_Cancer_(PAAD)'
cohort_dir = os.path.join(tcga_root_dir, selected_dir)

# Get the file paths for clinical and genetic data
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Extract standardized clinical features (Pancreatic_Cancer as the trait)
# In TCGA data, we use sample IDs to determine tumor vs. normal
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait="Pancreatic_Cancer",
    age_col=age_col, 
    gender_col=gender_col
)

# Save the clinical data to out_clinical_data_file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Saved clinical data with {len(clinical_features)} samples")

# Step 2: Normalize gene symbols in gene expression data
# Transpose to get genes as rows
gene_df = genetic_df

# Normalize gene symbols using NCBI Gene database synonyms
normalized_gene_df = normalize_gene_symbols_in_index(gene_df)
print(f"After normalization: {len(normalized_gene_df)} genes remaining")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene expression data")

# Step 3: Link clinical and genetic data
# Merge clinical features with genetic expression data
linked_data = clinical_features.join(normalized_gene_df.T, how='inner')
print(f"Linked data shape: {linked_data.shape} (samples x features)")

# Step 4: Handle missing values
cleaned_data = handle_missing_values(linked_data, trait_col="Pancreatic_Cancer")
print(f"After handling missing values, data shape: {cleaned_data.shape}")

# Step 5: Determine if trait or demographics are severely biased
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait="Pancreatic_Cancer")

# Step 6: Validate data quality and save cohort information
note = "The dataset contains gene expression data along with clinical information for pancreatic cancer patients."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=cleaned_data,
    note=note
)

# Step 7: Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Saved usable linked data to {out_data_file}")
else:
    print("Dataset was determined to be unusable and was not saved.")

Saved clinical data with 196 samples
After normalization: 19848 genes remaining


Saved normalized gene expression data
Linked data shape: (183, 19851) (samples x features)


After handling missing values, data shape: (183, 19851)
For the feature 'Pancreatic_Cancer', the least common label is '0' with 4 occurrences. This represents 2.19% of the dataset.
The distribution of the feature 'Pancreatic_Cancer' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 57.0
  50% (Median): 65.0
  75%: 73.0
Min: 35
Max: 88
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 82 occurrences. This represents 44.81% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset was determined to be unusable and was not saved.
